In [1]:
import numpy as np
import math

def gen():
    val = np.random.random((1,))
    if val < 2/3:
        return -1
    if val < 5/6:
        return 0
    return 1

def draw_u(d):
    return np.array([gen() for _ in range(d)])

def check_angle(mat, next, n, eps):
    for i in range(n):
        c = np.dot(mat[i], next) / np.linalg.norm(mat[i]) / np.linalg.norm(next)
        if abs(c) > eps:
            return True
    return False


def draw_set(k,d, eps):
    ans = np.zeros((k,d), np.float32)
    for i in range(k):
        next = draw_u(d)
        while check_angle(ans, next, i, eps):
            next = draw_u(d)
        ans[i] = next
    return ans


In [2]:
np.random.seed(0)
vectors = draw_set(6, 30, 0.1)
c = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        c[i][j] = np.dot(vectors[i], vectors[j]) / np.linalg.norm(vectors[i]) / np.linalg.norm(vectors[j])
print(c) 

np.round(c, decimals=3)

[[ 0.99999988 -0.03928371 -0.03928371 -0.04351941  0.07856742  0.04003204]
 [-0.03928371  1.00000012  0.03703704  0.08206099  0.         -0.03774257]
 [-0.03928371  0.03703704  1.00000012  0.0410305   0.          0.03774257]
 [-0.04351941  0.082061    0.0410305   1.         -0.0410305  -0.0418121 ]
 [ 0.07856742  0.          0.         -0.0410305   1.00000012  0.03774257]
 [ 0.04003204 -0.03774257  0.03774257 -0.0418121   0.03774257  1.        ]]


array([[ 1.   , -0.039, -0.039, -0.044,  0.079,  0.04 ],
       [-0.039,  1.   ,  0.037,  0.082,  0.   , -0.038],
       [-0.039,  0.037,  1.   ,  0.041,  0.   ,  0.038],
       [-0.044,  0.082,  0.041,  1.   , -0.041, -0.042],
       [ 0.079,  0.   ,  0.   , -0.041,  1.   ,  0.038],
       [ 0.04 , -0.038,  0.038, -0.042,  0.038,  1.   ]])

In [3]:
def draw_sample(n, seed = 0):
    u1 = vectors[0]
    u2 = vectors[1]
    u3 = vectors[2]
    u4 = vectors[3]
    u5 = vectors[4]
    u6 = vectors[5]
    f1 = lambda v1, v2, n: -2*u1 + v1*u2 -v2*u3 + n
    f2 = lambda v1, v2, n: -1.5*u4 -0.7*v1*u5 +0.9*v2*u6 + n
    f3 = lambda v1, v2, n: 1.2*u6 - v1*(u1+u2) +v2*u5 + n
    fs = [f1, f2, f3]

    np.random.seed(seed)
    ans = np.zeros((n*3,31))
    for i in range(n):
        for j in range(3):
            v1 = np.random.normal()
            v2 = np.random.normal()
            noise = np.random.standard_normal(30)*0.01
            ans[i*3+j][0:30] = fs[j](v1, v2, noise)
            ans[i*3+j][30] = j
    return ans        

In [4]:
class DataFrame(object):

    def __init__(self, train, test, n_train, n_test, d:int = 2):
        self.train = train
        self.test = test
        self.n_train = n_train
        self.n_test = n_test
        self.d = d

class DataSection(object):

    def __init__(self, data: DataFrame, train: bool, n:int):
        self.n = n
        chosen = data.train if train else data.test
        self.data = chosen[0:n,0:data.d]
        self.label = np.array(chosen[0:n,data.d], dtype=np.int32)


class KMean(DataSection):

    def __init__(self, data: DataFrame, train: bool, n:int, k:int, large = 1e6):
        super().__init__(data, train, n)
        self.k = k
        self.d = data.d
        self.large = large
        self.m = np.zeros((k,self.d), np.float32)
        self.assign = np.zeros((self.n), np.int32)
    
    def init_kpp(self, seed):
        np.random.seed(seed)
        choices = np.random.random((self.k,))
        prob = np.ones((self.n), np.float32)
        for i in range(self.k):
            choice = choices[i] * sum(prob)
            for j in range(self.n):
                choice -= prob[j]
                if choice <= 0:
                    self.m[i] = self.data[j]
                    break
            for j in range(self.n):
                min_dist = self.large
                for k in range(i+1):
                    diff = self.data[j] - self.m[k]
                    dist = np.dot(diff, diff)
                    min_dist = min(min_dist, dist)
                prob[j] = min_dist

    def init_random(self, seed):
        np.random.seed(seed)
        for i in range(self.k):
            self.m[i] = self.data[np.random.randint(self.n)]
                    
    def optimize_assign(self):
        sd = np.full((self.k, self.n, self.d), self.data)
        sm = np.full((self.n, self.k, self.d), self.m)
        sdf = sd - sm.swapaxes(0, 1)
        self.assign = np.argmin(np.linalg.norm(sdf, axis=2), 0)

    def test(self):
        diff = self.data - self.m[self.assign]
        return np.sum(np.square(diff)) / self.n

    def optimize_m(self):
        for i in range(self.k):
            self.m[i] = np.mean(self.data[self.assign == i],0)


In [5]:
data = DataFrame(draw_sample(200, 0), draw_sample(30000, 12345), 200, 1000, 30)

In [6]:
import matplotlib.pyplot as plt

def random_init(model: KMean, initializer, salt: int, n:int):
    best_init = 0
    best_loss = 1e6
    for i in range(n):
        initializer(model, salt + i)
        model.optimize_assign()
        loss = model.test()
        if loss < best_loss:
            best_loss = loss
            best_init = i
    initializer(model, salt + best_init)
    model.optimize_assign()

def train(model: KMean, eps: float):
    prev = 1e6
    loss_list = []
    loss = model.test()
    loss_list.append(loss)
    while prev-loss > eps:
        model.optimize_m()
        model.optimize_assign()
        prev = loss
        loss = model.test()
        loss_list.append(loss)
    return {"m":model.m, "loss":loss_list}

def test_and_list(m, n, k):
    model = KMean(data, False, n, k)
    model.m = m
    model.optimize_assign()
    table = np.zeros((3,k))
    partials = np.zeros((3,k))
    tot = [0,0,0]
    for i in range(3):
        for j in range(k):
            table[i,j] = np.mean(model.assign[model.label == i] == j)*100
            partials[i,j] = np.sum(model.assign[model.label == i] == j)
    for j in range(k):
        label = np.argmax(partials[:,j])
        for i in range(3):
            if i == label:
                continue
            tot[i] += partials[i,j]
    return {"table":table,"total":tot,"test":model}

In [7]:
def train_and_test(k, func, salt = 123456, eps = 1e-4, n_init = 10):
    model = KMean(data, True, 600, k)
    random_init(model, func, salt + k, n_init)
    result = train(model, eps)
    tables = test_and_list(result["m"], 90000, k)
    return {"m":result["m"], "loss":result["loss"], "table":tables["table"], "total":tables["total"], "model":model, "test":tables["test"]}

In [8]:
results = [train_and_test(k, KMean.init_kpp, 52435) for k in range(2,7)]

In [9]:
for k in range(2,7):
    r = results[k-2]
    print(np.round(r["table"], decimals=1))

[[100.    0. ]
 [  0.  100. ]
 [ 19.1  80.9]]
[[  0.  100.    0. ]
 [ 86.7   0.   13.3]
 [  0.   15.3  84.7]]
[[  0.    0.    0.  100. ]
 [ 10.2  87.9   1.9   0. ]
 [ 55.    0.   43.9   1.1]]
[[62.6 37.4  0.   0.   0. ]
 [ 0.   0.  84.4  0.  15.6]
 [ 0.  10.9  0.  38.2 50.9]]
[[38.3 31.   0.   0.   0.  30.7]
 [ 0.   0.  84.8  0.  15.2  0. ]
 [ 0.  10.2  0.  38.8 50.9  0. ]]


In [10]:
for k in range(2,7):
    r = results[k-2]
    print(np.round(np.array(r["total"])/30000*100,1))
    print(np.round(np.mean(np.array(r["total"])/30000*100),1))

[  0.   0. 100.]
33.3
[ 0.  13.3 15.3]
9.5
[ 0.  12.1  1.1]
4.4
[ 0.  15.6 10.9]
8.8
[ 0.  15.2 10.2]
8.5


In [11]:
for k in range(2,7):
    r = results[k-2]
    cov = np.zeros((6,k))
    for i in range(6):
        for j in range(k):
            a = vectors[i]
            b = r["m"][j]
            cov[i,j] = np.dot(a,b)/np.linalg.norm(a)/np.linalg.norm(b)
    print(np.round(cov, decimals=3))

[[-0.99   0.169]
 [-0.068  0.026]
 [ 0.028 -0.01 ]
 [ 0.032 -0.797]
 [-0.101  0.175]
 [ 0.05   0.604]]
[[ 0.042 -0.992  0.182]
 [-0.075 -0.06   0.092]
 [-0.047  0.028  0.03 ]
 [-0.985  0.033 -0.182]
 [ 0.102 -0.098  0.141]
 [-0.119  0.039  0.965]]
[[-0.235  0.045  0.424 -1.   ]
 [-0.318 -0.076  0.36   0.02 ]
 [ 0.027 -0.046  0.027  0.027]
 [-0.189 -0.986 -0.048  0.041]
 [-0.242  0.137  0.428 -0.077]
 [ 0.867 -0.091  0.738 -0.031]]
[[-0.955 -0.821  0.043  0.513 -0.142]
 [ 0.284 -0.471 -0.074  0.48  -0.251]
 [ 0.209 -0.231 -0.047  0.026  0.026]
 [ 0.068 -0.017 -0.981 -0.013 -0.264]
 [-0.074 -0.079  0.12   0.2    0.001]
 [-0.041  0.081 -0.131  0.685  0.933]]
[[-0.897 -0.812  0.043  0.509 -0.153 -0.841]
 [ 0.428 -0.47  -0.074  0.475 -0.261  0.026]
 [-0.155 -0.263 -0.047  0.026  0.026  0.573]
 [ 0.063 -0.019 -0.982 -0.013 -0.258  0.056]
 [-0.07  -0.077  0.124  0.205 -0.012 -0.065]
 [-0.058  0.08  -0.126  0.692  0.929 -0.011]]


In [12]:
class EMAlg(DataSection):

    def __init__(self, data: DataFrame, train: bool, n:int, k:int, d:int):
        super().__init__(data, train, n)
        self.k = k
        self.d = d
        self.m = np.zeros((k,d), np.float32)
        self.c = np.full((k,d,d), np.identity(d), np.double)
        self.pi = np.full((k,), 1/k, np.double)
        self.assign = np.zeros((k, self.n), np.double)
        self.logprob = np.zeros((k, self.n), np.double)

    def step_assignment(self):
        self.logprob = np.zeros((self.k, self.n), np.double)
        for i in range(self.k):
            c = self.c[i]
            m = self.m[i]
            ic = np.linalg.inv(c)
            factor = math.sqrt(abs(np.linalg.det(c))) # * (2 * math.pi) ** (self.d/2)
            logfactor = np.log(factor)
            logpi = np.log(self.pi[i])
            for j in range(self.n):
                x = self.data[j]
                upper = np.matmul(np.matmul(x - m, ic), x - m)
                self.logprob[i,j] = -upper / 2 - logfactor + logpi # constant offset
        for j in range(self.n):
            vec = self.logprob[:,j]
            avg = np.mean(vec)
            bound = np.max(np.abs(vec-avg))
            if bound > 100:
                vec = np.exp((vec-avg)/bound*100)
            else:
                vec = np.exp(vec-avg)
            total = sum(vec)
            for i in range(self.k):
                self.assign[i,j] = vec[i] / total # no offset
        
    
    def step_update(self):
        for i in range(self.k):
            neff = sum(self.assign[i])
            m = np.zeros((self.d,))
            for j in range(self.n):
                m += self.assign[i,j]*self.data[j]
            self.m[i] = m/neff
            c = np.zeros((self.d, self.d))
            for j in range(self.n):
                diff = self.data[j] - self.m[i]
                c += self.assign[i,j] * np.outer(diff, diff)   
            self.c[i] = c/neff
            self.pi[i] = neff / self.n

    def test(self):
        sa = np.sum(self.assign * self.logprob) # constant offset
        sc = np.sum(self.pi)*self.n
        return sa - sc

In [13]:
def test_em(old: EMAlg, n, k):
    model = EMAlg(data, False, n, k, 30)
    model.m = np.copy(old.m)
    model.c = np.copy(old.c)
    model.pi = np.copy(old.pi)
    model.step_assignment()
    table = np.zeros((3,k))
    assign = np.argmax(model.assign, 0)
    partials = np.zeros((3,k))
    tot = [0,0,0]
    for i in range(3):
        for j in range(k):
            table[i,j] = np.mean(assign[model.label == i] == j)*100
            partials[i,j] = np.sum(assign[model.label == i] == j)
    for j in range(k):
        label = np.argmax(partials[:,j])
        for i in range(3):
            if i == label:
                continue
            tot[i] += partials[i,j]
    return {"table":table,"total":tot,"test":model}

def gen_and_test_em(base: KMean, n, k):
    emalg = EMAlg(data, True, n, k, 30)
    emalg.m = np.copy(base.m)
    emalg.step_assignment()

    prev = 1e6
    val_list = []
    val = emalg.test()
    val_list.append(val)
    while prev - val > 1e-4:
        emalg.step_update()
        emalg.step_assignment()
        prev = val
        val = emalg.test()
        val_list.append(val)

    tested = test_em(emalg, 90000, k)
    mat = np.round(tested["table"], decimals=1)
    print(mat)
    print(tested["total"])

    return {"model":emalg,"list":val_list,"mat":mat,"total":tested["total"]}


In [17]:

for k in range(2,7):
    results[k-2]["em"]=gen_and_test_em(results[k-2]["model"],600,k)

[[100.    0. ]
 [  0.  100. ]
 [  1.9  98.1]]
[0.0, 0.0, 30000.0]
[[  0.  100.    0. ]
 [ 99.9   0.    0.1]
 [  0.    0.3  99.7]]
[0.0, 17.0, 81.0]
[[  0.1   0.    0.   99.9]
 [  0.  100.    0.    0. ]
 [  0.2   0.   10.4  89.4]]
[37.0, 1.0, 26826.0]
[[99.3  0.7  0.   0.   0. ]
 [ 0.   0.  99.8  0.   0.2]
 [ 0.   1.4  0.  58.4 40.2]]
[218.0, 62.0, 0.0]
[[60.2  4.8  0.   0.   0.  35. ]
 [ 0.   0.  99.8  0.   0.2  0. ]
 [ 0.   1.4  0.  58.9 39.7  0. ]]
[0.0, 58.0, 411.0]


In [22]:
for k in range(2,7):
    print(np.round(np.mean(np.array(results[k-2]["em"]['total']))/300,1))

33.3
0.1
29.8
0.3
0.5


In [45]:
def cov_find(model: EMAlg, _k):
    cor = np.zeros((_k,6,30))
    for i in range(_k):
        w,v = np.linalg.eig(model.c[i])
        for j in range(6):
            v0 = vectors[j]
            for k in range(30):
                v1 = v[:,k]
                cor[i,j,k] = np.abs(np.dot(v0,v1)/np.linalg.norm(v0)/np.linalg.norm(v1))*w[k]
    return cor

for k in range(2,7):
    print(np.round(np.max(cov_find(results[k-2]["em"]["model"], k),2),0))


        

[[ 4. 30. 28.  3.  2.  7.]
 [ 7.  8.  1. 15. 18. 22.]]
[[ 1.  0.  0.  1. 11. 10.]
 [ 3. 30. 28.  3.  2.  7.]
 [12. 12.  0.  6. 18.  1.]]
[[28. 12.  4. 12. 13. 16.]
 [ 1.  1.  1.  1. 12. 20.]
 [25. 13.  3. 12. 15. 17.]
 [31. 21. 24.  2.  8. 16.]]
[[ 1. 15. 31.  1.  0.  2.]
 [ 1. 13. 19.  1.  2.  8.]
 [ 1.  0.  0.  1. 10.  9.]
 [ 8.  8.  0.  1. 27.  1.]
 [ 5.  4.  0.  8. 20.  2.]]
[[ 1. 11. 13.  1.  0.  1.]
 [ 1. 13. 17.  1.  2.  9.]
 [ 1.  0.  0.  1.  9. 10.]
 [ 8.  8.  0.  1. 27.  1.]
 [ 5.  4.  0.  8. 20.  2.]
 [ 1. 15.  9.  1.  0.  1.]]
